In [ ]:
# ============================================================
# BioCUP — End-to-end Search Test (multi-chunks -> % primary_site)
# Compatible with qdrant-client where query_points uses: query=..., using="dense"/"sparse"
# ============================================================

import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from collections import defaultdict
from qdrant_client import QdrantClient
from qdrant_client.http import models as qm

# ----------------------------
# 0) CONFIG
# ----------------------------
load_dotenv()

COLLECTION = "biocup_hybrid_splade_v1"

# Input embeddings (3 files)
INPUT_EMB_DIR = os.path.normpath("../data/input/embeddings")  # adapte si besoin
META_PATH  = os.path.join(INPUT_EMB_DIR, "meta.parquet")        # ou meta.csv
DENSE_PATH = os.path.join(INPUT_EMB_DIR, "dense.npy")
SPARSE_PATH = os.path.join(INPUT_EMB_DIR, "sparse_splade.npz")

# Search params
K_DENSE = 40
K_SPARSE = 40
K_FUSED = 30
RRF_K = 60

# Evidence per predicted site
MAX_EVIDENCE_PER_SITE = 5

# Section weights (tunable)
SECTION_WEIGHT = {
    "IHC": 1.5,
    "DIAGNOSIS": 1.3,
    "SYNOPTIC": 1.2,
    "LYMPH_NODES": 1.0,
    "LYMPH": 1.0,
    "MARGINS": 1.0,
    "MICRO": 0.9,
    "GROSS": 0.7,
    "COMMENT": 0.6,
    "GENERAL": 0.6,
}

def w_section(section: str) -> float:
    return SECTION_WEIGHT.get((section or "GENERAL").upper(), 1.0)

# ----------------------------
# 1) CONNECT QDRANT
# ----------------------------
client = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ.get("QDRANT_API_KEY")
)

# quick health check
info = client.get_collection(COLLECTION)
print("Collection status:", info.status, "points:", info.points_count)

# ----------------------------
# 2) LOAD INPUT EMBEDDINGS
# ----------------------------
def read_meta(path: str) -> pd.DataFrame:
    if path.endswith(".parquet"):
        return pd.read_parquet(path)
    return pd.read_csv(path)

meta = read_meta(META_PATH)
dense = np.load(DENSE_PATH)
sp = np.load(SPARSE_PATH, allow_pickle=True)
sp_indices = sp["indices"]
sp_values = sp["values"]

N = len(meta)
assert dense.shape[0] == N == len(sp_indices) == len(sp_values), "❌ Input files misaligned"
print("✅ Loaded input chunks:", N, "dense shape:", dense.shape)

# ----------------------------
# 3) FILTERS (light guidance only)
# ----------------------------
def build_light_filter(chunk_section: str) -> qm.Filter:
    must = []
    must_not = [qm.FieldCondition(key="is_admin_noise", match=qm.MatchValue(value=1))]

    sec = (chunk_section or "").upper()
    if sec == "IHC":
        must.append(qm.FieldCondition(key="has_ihc", match=qm.MatchValue(value=True)))
    elif sec in ("LYMPH_NODES", "LYMPH"):
        must.append(qm.FieldCondition(key="has_lymph", match=qm.MatchValue(value=True)))
    elif sec == "MARGINS":
        must.append(qm.FieldCondition(key="has_margins", match=qm.MatchValue(value=True)))

    return qm.Filter(must=must, must_not=must_not)

# ----------------------------
# 4) RRF FUSION (hybrid in code)
# ----------------------------
def rrf_fuse(dense_points, sparse_points, k=RRF_K):
    """
    Rank-based fusion: sum 1/(k+rank+1)
    Returns: fused_list (PointStruct-like), id2score, id2payload
    """
    id2score = {}
    id2payload = {}

    for rank, p in enumerate(dense_points):
        pid = p.id
        id2score[pid] = id2score.get(pid, 0.0) + 1.0/(k + rank + 1)
        if pid not in id2payload and p.payload is not None:
            id2payload[pid] = p.payload

    for rank, p in enumerate(sparse_points):
        pid = p.id
        id2score[pid] = id2score.get(pid, 0.0) + 1.0/(k + rank + 1)
        if pid not in id2payload and p.payload is not None:
            id2payload[pid] = p.payload

    fused_ids = [pid for pid, _ in sorted(id2score.items(), key=lambda x: x[1], reverse=True)]
    return fused_ids, id2score, id2payload

# ----------------------------
# 5) END-TO-END SEARCH + AGGREGATION
# ----------------------------
scores_by_site = defaultdict(float)
evidence_by_site = defaultdict(list)

# prevent the same case from over-voting inside the SAME input chunk
best_case_contrib_per_chunk = {}  # (chunk_i, case_id) -> best score

for i in range(N):
    row = meta.iloc[i]
    sec = str(row.get("section", "GENERAL"))
    sec_w = w_section(sec)

    q_dense = dense[i].tolist()
    q_sparse = qm.SparseVector(
        indices=sp_indices[i].tolist(),
        values=sp_values[i].tolist()
    )

    flt = build_light_filter(sec)

    # ---- DENSE QUERY (your version: query=..., using="dense")
    dense_res = client.query_points(
        collection_name=COLLECTION,
        query=q_dense,
        using="dense",
        query_filter=flt,
        limit=K_DENSE,
        with_payload=True
    )
    dense_points = dense_res.points or []

    # ---- SPARSE QUERY (your version: query=SparseVector, using="sparse")
    sparse_res = client.query_points(
        collection_name=COLLECTION,
        query=q_sparse,
        using="sparse",
        query_filter=flt,
        limit=K_SPARSE,
        with_payload=True
    )
    sparse_points = sparse_res.points or []

    # ---- HYBRID (RRF fusion)
    fused_ids, id2rrf, id2payload = rrf_fuse(dense_points, sparse_points, k=RRF_K)
    fused_ids = fused_ids[:K_FUSED]

    # ---- Aggregate votes by primary_site
    for pid in fused_ids:
        payload = id2payload.get(pid)
        if not payload:
            continue

        site = payload.get("primary_site")
        case_id = payload.get("case_id")
        if not site or not case_id:
            continue

        contrib = id2rrf.get(pid, 0.0) * sec_w

        key = (i, str(case_id))
        if contrib <= best_case_contrib_per_chunk.get(key, 0.0):
            continue
        best_case_contrib_per_chunk[key] = contrib

        scores_by_site[site] += contrib

        # keep evidence snippets
        if len(evidence_by_site[site]) < MAX_EVIDENCE_PER_SITE:
            txt = payload.get("chunk_text", "")
            snippet = (txt[:180] + "...") if isinstance(txt, str) and len(txt) > 180 else txt
            evidence_by_site[site].append({
                "score": contrib,
                "case_id": str(case_id),
                "section": payload.get("section"),
                "snippet": snippet
            })

# ----------------------------
# 6) NORMALIZE -> % BY SITE
# ----------------------------
total = sum(v for v in scores_by_site.values() if v > 0)
percent_by_site = {k: (v/total)*100.0 for k, v in scores_by_site.items()} if total > 0 else {}

# sort results
sorted_sites = sorted(percent_by_site.items(), key=lambda x: x[1], reverse=True)

print("\n==============================")
print("✅ Predicted primary_site (%)")
print("==============================")
for site, pct in sorted_sites[:10]:
    print(f"{site:15s} {pct:6.2f}%")

# show evidence for top-1 and top-3
print("\n==============================")
print("🔎 Evidence (top sites)")
print("==============================")
for site, pct in sorted_sites[:3]:
    print(f"\n--- {site} ({pct:.2f}%) ---")
    ev = sorted(evidence_by_site[site], key=lambda d: d["score"], reverse=True)
    for e in ev:
        print(f"  score={e['score']:.4f} case={e['case_id']} sec={e['section']} | {e['snippet']}")


Collection status: green points: 5104


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\data\\input\\embeddings\\meta.parquet'